In [1]:
import jsonlines
import pandas as pd
import emoji
from soynlp.normalizer import repeat_normalize

In [2]:
from tqdm import tqdm
import selenium
from selenium import webdriver
from selenium.webdriver.remote.webdriver import WebDriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import urllib
import re

# Backtranslation

In [61]:
import urllib.parse

def translation(df, src_lang, tgt_lang, driver: WebDriver):
    """
    request format: sk={src_lang}&tk={tgt_lang}&st={text}
    """
    
    request_url = 'https://papago.naver.com/?sk={}&tk={}&st={}'
    is_formal = False
    augmented_texts = []
    for idx, row in tqdm(df.iterrows(), total=len(df)):
        text = row['title']
        
        translated_text = ""
        while translated_text == "": # 번역이 제대로 되지 않았을 경우 다시 번역
            try:
                translated_text = urllib.parse.quote(translated_text) # 특수문자로 인한 encoding
                driver.get(request_url.format(src_lang, tgt_lang, text))
                driver.implicitly_wait(60)
                time.sleep(1.0)

                if tgt_lang == 'ko' and is_formal: # 한국어로 번역할 경우 높임말 해제 (only for en, ja, cn)
                    driver.find_element(By.XPATH, '//*[@id="root"]/div/div[1]/section/div/div[1]/div[3]/div/div[6]/div/button').click()
                    is_formal = False
                    time.sleep(1.5)
                translated_text = driver.find_element(By.XPATH, '//*[@id="txtTarget"]').text # translation result

            except: # 오류 발생 시 get 방식이 아닌 직접 텍스트를 입력하도록 시도
                driver.get(request_url.format(src_lang, tgt_lang, ''))
                time.sleep(1.5)
                driver.find_element(By.XPATH, '//*[@id="txtSource"]').send_keys(text)
                time.sleep(1.5)
                translated_text = driver.find_element(By.XPATH, '//*[@id="txtTarget"]').text

        assert translated_text != ""
        augmented_texts.append(translated_text)

    df['title'] = augmented_texts
    return df


In [63]:
def augmentation_backtranslate(tgt_lang, file_in, file_out):
    
    df = pd.read_csv(file_in)
    print(len(df))

    options = webdriver.ChromeOptions()
    options.add_argument('headless')
    # options.add_argument('Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36')
    driver = webdriver.Chrome(options=options)
    forward = translation(df, 'ko', tgt_lang, driver)
    backward = translation(forward, tgt_lang, 'ko', driver)

    print(len(backward))

    driver.close()

    # assert len(text_data) == len(backward)

    backward.to_csv(file_out, index=False)

    return forward, backward

In [64]:
# (대상 언어, 입력 파일명, 출력 파일명)
forward, backward = augmentation_backtranslate('ja', 'data/train_low.csv', 'data/train_bt_ja.csv')

280


100%|██████████| 280/280 [08:07<00:00,  1.74s/it]


280


# Postprocess


### Check N/A

In [59]:
train_df = pd.read_csv('data/train_low.csv')

### Merge multiple augmented files

In [65]:
files = [
    'data/train_bt_en.csv',
    'data/train_bt_ja.csv',
    'data/train_bt_cn.csv',
]

aug_df = pd.concat([pd.read_csv(file) for file in files])
aug_df.drop_duplicates(subset=['title'], inplace=True)
aug_df

,guid,title,label
0,ynat-v1_train_20804,두산 정수빈은 군 복무 후 마지막 안타 때까지 용병급 활약을 펼쳤다,5
1,ynat-v1_train_40850,"하나금융투자, IT서비스 국제인증 획득",0
2,ynat-v1_train_15259,한국방송대학교 청각장애학생을 위한 미디어강좌 부제 서비스 제공,2
3,ynat-v1_train_37602,니카라과 망명 중인 엘살바도르 전 대통령 시민권,4
4,ynat-v1_train_23775,신우인 목사의 이스라엘 왕 이야기,3
...,...,...,...
275,ynat-v1_train_12293,독일 프리랜서 기자 세월호 다큐멘터리 베를린 등서 상영,4
276,ynat-v1_train_20900,질의응답 한인섭 한국형사정책연구원장,2
277,ynat-v1_train_22610,이란군 최신 구축함 등 해군 함대 3월부터 5개월간 대서양 항해,4
278,ynat-v1_train_10964,"김학의 법원행정처장 재정신청 기각, 더 이상 볼 일 아니다",2


In [66]:
# Train data에 있는 데이터는 제외
aug_df = aug_df[~aug_df['title'].isin(train_df['title'])]
aug_df

,guid,title,label
0,ynat-v1_train_20804,두산 정수빈은 군 복무 후 마지막 안타 때까지 용병급 활약을 펼쳤다,5
1,ynat-v1_train_40850,"하나금융투자, IT서비스 국제인증 획득",0
2,ynat-v1_train_15259,한국방송대학교 청각장애학생을 위한 미디어강좌 부제 서비스 제공,2
3,ynat-v1_train_37602,니카라과 망명 중인 엘살바도르 전 대통령 시민권,4
4,ynat-v1_train_23775,신우인 목사의 이스라엘 왕 이야기,3
...,...,...,...
275,ynat-v1_train_12293,독일 프리랜서 기자 세월호 다큐멘터리 베를린 등서 상영,4
276,ynat-v1_train_20900,질의응답 한인섭 한국형사정책연구원장,2
277,ynat-v1_train_22610,이란군 최신 구축함 등 해군 함대 3월부터 5개월간 대서양 항해,4
278,ynat-v1_train_10964,"김학의 법원행정처장 재정신청 기각, 더 이상 볼 일 아니다",2


In [71]:
num_aug = 2
# sample num_aug data from each guid
aug_df = aug_df.groupby('guid').head(num_aug)
aug_df['guid'].value_counts()

ynat-v1_train_00401    2
ynat-v1_train_30431    2
ynat-v1_train_32512    2
ynat-v1_train_31643    2
ynat-v1_train_31219    2
                      ..
ynat-v1_train_16521    2
ynat-v1_train_16652    2
ynat-v1_train_16736    2
ynat-v1_train_16784    2
ynat-v1_train_45546    2
Name: guid, Length: 280, dtype: int64

In [74]:
aug_df = pd.concat([train_df, aug_df])
aug_df.sort_values(by=['guid'], inplace=True, kind='mergesort')
aug_df.to_csv('data/train_bt_2.csv', index=False)